# Using a LLM such as bert to give a basic summary of poeple opinions of the cars



In [1]:
import pandas as pd

import os
import spacy 
from spacy import displacy

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [2]:
# Trying to read with more flexible options
try:
    # Attempt with error handling options
    df = pd.read_csv("./car_review_datasets/Scraped_Car_Review_ford.csv", 
                     error_bad_lines=False,  # Skip bad lines
                     warn_bad_lines=True,    # Warn about them
                     encoding='utf-8')       # Explicit encoding
except:
    # If that doesn't work, try a different approach
    df = pd.read_csv("./car_review_datasets/Scraped_Car_Review_ford.csv", 
                     engine='python',        # Python engine is more flexible
                     encoding='utf-8')       # Explicit encoding

In [3]:
df = df.drop('Unnamed: 0', axis=1)
df_fiesta = df[df["Vehicle_Title"].str.contains("Fiesta", na=False)]
df_fiesta["vehicle_age"] = df_fiesta["Vehicle_Title"].str.extract(r'(\d{4})')[0]
df_fiesta["vehicle_age"] = pd.to_numeric(df_fiesta["vehicle_age"])
df_fiesta = df_fiesta[df_fiesta["vehicle_age"] < 2017]
df_fiesta = df_fiesta.reset_index(drop=True)

/tmp/ipykernel_411301/3309848676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fiesta["vehicle_age"] = df_fiesta["Vehicle_Title"].str.extract(r'(\d{4})')[0]
/tmp/ipykernel_411301/3309848676.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fiesta["vehicle_age"] = pd.to_numeric(df_fiesta["vehicle_age"])


In [4]:
df_fiesta_reviews = df_fiesta[["Review", "Rating", "Review_Title", "vehicle_age"]].copy()


In [18]:
df_fiesta_reviews[0:5]["Review"].values

array([' We bought a Ford Fiesta 2014 in March. The Sound was Popping on and off for the longest time. We took it to the Ford dealership 2x visited 2x to complain of loud popping from the radio. They kept saying "The diagnostics didn\'t show anything". Now our radio doesn\'t come on at all the sound makes high pitch noises, scratchy noises, and terrible. We went to a car machanic guy and he says "Yeah, he had 3-4 of the same vehicles with this issue." He said it is the chip in it which costs $500 to fix."  If you\'re looking into this vehicle please note there is also no physical Locks on the doors either which could possibly mess up and should be considered a safety hazard.   We have had this car for only 10 months and is MORE TROUBLE than it is Worth.',
       ' OK, I will be honest, at first I didn\'t like the quirky transmission of the car. It NEVER did anything serious such as stall, fail to make the car go when I stepped on the accelerator or some of the other things I have seen 

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from transformers import pipeline


/home/reaf/.cache/pypoetry/virtualenvs/mot-analysis-129goUNT-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
df_fiesta_reviews = df_fiesta[["Review", "Rating", "Review_Title", "vehicle_age"]].copy()


In [27]:
pos_reviews = df_fiesta_reviews[df_fiesta_reviews["Rating"]> 3]

In [33]:
summarizer = pipeline('summarization', model="facebook/bart-large-cnn")


stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return tokens


nlp = spacy.load('en_core_web_sm')
def lemmatize(tokens):
    doc = nlp(' '.join(tokens))
    return [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB']]


Device set to use cpu


In [ ]:
def batch_summarize(reviews, sentiment="positive", batch_size=5):
    """
    Summarize reviews in batches to avoid memory issues.
    """

    all_reviews = reviews['Review'].tolist()

    total_reviews = len(reviews)
    batches = []

    # Process reviews in smaller batches
    for i in range(0, total_reviews, batch_size):
        batch = all_reviews[i:i + batch_size]
        batches.append(batch)
    
    print(f"Processing {total_reviews} {sentiment} reviews in {len(batches)} batches...")
    
    # Process each batch separately
    batch_summaries = []
    for i, batch in enumerate(batches):
        print(f"Summarizing batch {i+1}/{len(batches)}...")
        
        # Generate summaries for each review in the batch
        batch_review_summaries = []
        for review in batch:
            if isinstance(review, str) and len(review.strip()) > 100:
                # Truncate long reviews to avoid token limits
                truncated_review = review[:3500]
                try:
                    summary = summarizer(truncated_review, max_length=100, min_length=30, do_sample=False)
                    batch_review_summaries.append(summary[0]['summary_text'])
                except Exception as e:
                    print(f"Error processing review: {e}")
        
        # Combine the summaries from this batch
        if batch_review_summaries:
            combined_batch = " ".join(batch_review_summaries)
            # Generate a summary of the combined summaries
            try:
                batch_summary = summarizer(combined_batch, max_length=200, min_length=50, do_sample=False)
                batch_summaries.append(batch_summary[0]['summary_text'])
            except Exception as e:
                print(f"Error summarizing batch: {e}")
        
    final_summary_text = " ".join(batch_summaries)
    
    # Generate the final comprehensive summary
    try:
        final_summary = summarizer(final_summary_text, 
                                   max_length=250, 
                                   min_length=150, 
                                   do_sample=False)
        
        print(f"Final summary generated for {sentiment} reviews.")
        print("=" * 70)
        print(final_summary[0]['summary_text'])
        print("=" * 70)

        return final_summary[0]['summary_text']
    except Exception as e:
        print(f"Error generating final summary: {e}")
        return "Failed to generate comprehensive summary due to an error."



def create_full_summary(df_fiesta_reviews):
    """
    Create a comprehensive executive summary of Ford Fiesta reviews
    using a multi-stage summarization approach.
    """

    print("Generating comprehensive Ford Fiesta executive summary...")



    pos_reviews = df_fiesta_reviews[df_fiesta_reviews["Rating"]> 3].sample(n=20)
    neg_reviews = df_fiesta_reviews[df_fiesta_reviews["Rating"]< 3].sample(n=20)

    try:
        final_pos_summary = batch_summarize(pos_reviews, sentiment="positive", batch_size=5)
        final_neg_summary = batch_summarize(neg_reviews, sentiment="negative", batch_size=5)

        contrastive_prompt = (
                f"Positive summary: {final_pos_summary}\n\n" +
                f"Negative summary: {final_neg_summary}\n\n" +
                "Based on the contrasting opinions above about the Ford Fiesta, provide a balanced perspective on the car's strengths and weaknesses."
            )
        
        print(contrastive_prompt)
        
        contrastive_result = summarizer(contrastive_prompt[:3500], max_length=200, min_length=100, do_sample=False)

        return contrastive_result[0]['summary_text']
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Failed to generate comprehensive summary due to an error."
    

contrastive_summary = create_full_summary(df_fiesta_reviews)

print("\n\nCONTRASTIVE SUMMARY OF FORD FIESTA REVIEWS:")
print("="*70)
print(contrastive_summary)
print("="*70)








        


Generating comprehensive Ford Fiesta executive summary...
Processing 20 positive reviews in 4 batches...
Summarizing batch 1/4...


Your max_length is set to 100, but your input_length is only 52. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)
Your max_length is set to 100, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)


Summarizing batch 2/4...
Summarizing batch 3/4...


Your max_length is set to 100, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


Summarizing batch 4/4...


...

Summarizing batch 2/4...


Your max_length is set to 100, but your input_length is only 58. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)


Summarizing batch 3/4...


Your max_length is set to 100, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 100, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)


Summarizing batch 4/4...


CONTRASTIVE SUMMARY OF FORD FIESTA REVIEWS:
The 2016 Fiesta SE Sedan is our new RV motorhome tow car. Good road handling, good acceleration, sport car feel steering, great visibility, incredible gas mileage. With a base price of $13,900 it's a great value. The hatchback is the way to go when it comes to style, function, and long term value. For a LITTLE car, this vehicle has been quite functional and comfortable. Upholstery is in great shape even after 2 years of beach trips.
